<a href="https://colab.research.google.com/github/mdorodch/ITSC3155-Demos/blob/master/Hour2_RL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import gym
import random

In [ ]:
#env = gym.make("FrozenLake-v1")
desc=["SFFF", "FHFH", "FFFH", "HFFG"]
env= gym.make('FrozenLake-v1', desc=desc, map_name="4x4", is_slippery=False)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


##Frozen Lake environment
https://www.gymlibrary.dev/environments/toy_text/frozen_lake/

Reward schedule:

Reach goal(G): +1     

Reach hole(H): 0

Reach frozen(F): 0

In [ ]:
print(env.action_space)
print(env.observation_space)
#desc=["SFFF", "FHFH", "FFFH", "HFFG"]

Discrete(4)
Discrete(16)


In [ ]:
# Create our Q table with state_size rows and action_size columns (16x4)
action_size = env.action_space.n
state_size = env.observation_space.n
print(state_size, action_size)

qtable = np.zeros((state_size, action_size))
print(qtable)

16 4
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


##Q-learning
Choose an action in state s, observe the reward, and update Q-value associated with (s,a)

In [ ]:
total_episodes = 1000
max_steps = 100  #try maximum 100 steps inside a game, to avoid a loop
epsilon = 0.5    # Exploration rate
learning_rate = 0.7    # Learning rate
gamma = 0.95                 # Discounting rate

# Exploration parameters
epsilon = 1.0                 # Exploration rate
max_epsilon = 1.0             # Exploration probability at start
min_epsilon = 0.01            # Minimum exploration probability
decay_rate = 0.005            # Exponential decay rate for exploration prob

In [ ]:
def q_learning(episodes,epsilon):
  rewards = []
  # Each episode happen after a game ends (agent succeed or die)
  for episode in range(episodes):
      # Reset the environment
      state = env.reset()
      step = 0
      done = False
      total_rewards = 0
      if episode % 1000 == 0:
          print("Episode: ", episode)

      # The Q-Table learning algorithm
      for step in range(max_steps):
          # 3. Choose an action a in the current world state (s)
          ## First we randomize a number
          exploit_rate = random.uniform(0, 1)

          ## If this number > 0.2, exploit
          if exploit_rate > epsilon:
              action = np.argmax(qtable[state,:])
              #print(exp_exp_tradeoff, "action", action)
          # Else doing a random choice --> exploration
          else: #explore
              action = env.action_space.sample()
              #print("action random", action)

          # Take the action (a) and observe the outcome state(s') and reward (r)
          new_state, reward, done, info = env.step(action)
          qtable[state, action] = qtable[state, action] + learning_rate * (reward + gamma * np.max(qtable[new_state, :]) - qtable[state, action])

          total_rewards += reward
          state = new_state  #update the state

          # If done (if we're dead) : finish episode
          if done == True:
              break

      # Reduce learning rate, to converge
      #learning_rate = 1/(episode+1)
      epsilon = min_epsilon + (max_epsilon - min_epsilon)*np.exp(-decay_rate*episode)
      rewards.append(total_rewards)

  print ("Average score over episodes: " +  str(sum(rewards)/total_episodes))
  print(qtable)

q_learning(1000,epsilon)
#action 0: LEFT, 1: DOWN, 2: RIGHT, 3: UP
#state = current_row * nrows + current_col

Episode:  0
Average score over episodes: 0.0
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


## Test performance

In [ ]:
test_episodes = 10
def test_agent(test_episodes):
  for episode in range(test_episodes):
      state = env.reset()
      step = 0
      done = False
      print("****************************************************")
      print("EPISODE ", episode)

      for step in range(max_steps):
          # Take the action (index) that have the maximum expected future reward given that state
          action = np.argmax(qtable[state,:])
          new_state, reward, done, info = env.step(action)
          if done:
              # Here, we decide to only print the last state (to see if our agent is on the goal or fall into an hole)
              #env.render()
              if new_state == 15:
                  print("We reached our Goal 🏆")
              else:
                  print("We fell into a hole ☠️")

              # We print the number of step it took.
              print("Number of steps", step)

              break
          state = new_state
  env.close()

test_agent(test_episodes)

****************************************************
EPISODE  0
We fell into a hole ☠️
Number of steps 99
****************************************************
EPISODE  1
We fell into a hole ☠️
Number of steps 99
****************************************************
EPISODE  2
We fell into a hole ☠️
Number of steps 99
****************************************************
EPISODE  3
We fell into a hole ☠️
Number of steps 99
****************************************************
EPISODE  4
We fell into a hole ☠️
Number of steps 99
****************************************************
EPISODE  5
We fell into a hole ☠️
Number of steps 99
****************************************************
EPISODE  6
We fell into a hole ☠️
Number of steps 99
****************************************************
EPISODE  7
We fell into a hole ☠️
Number of steps 99
****************************************************
EPISODE  8
We fell into a hole ☠️
Number of steps 99
**********************************************

In [ ]:
q_learning(10000,epsilon)

Episode:  0
Episode:  1000
Episode:  2000
Episode:  3000
Episode:  4000
Episode:  5000
Episode:  6000
Episode:  7000
Episode:  8000
Episode:  9000
Average score over episodes: 9.688
[[0.73509189 0.6983373  0.77378094 0.73509189]
 [0.73509189 0.         0.81450625 0.77378094]
 [0.77378094 0.857375   0.77378094 0.81450625]
 [0.81450625 0.         0.67956691 0.6267943 ]
 [0.69829417 0.65757668 0.         0.73509189]
 [0.         0.         0.         0.        ]
 [0.         0.9025     0.         0.81450625]
 [0.         0.         0.         0.        ]
 [0.45127704 0.         0.56288833 0.6983373 ]
 [0.66342043 0.         0.63175    0.        ]
 [0.63024941 0.95       0.         0.857375  ]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.         0.86542513 0.95       0.44117112]
 [0.9025     0.95       1.         0.9025    ]
 [0.         0.         0.         0.        ]]


In [ ]:
test_agent(test_episodes)

****************************************************
EPISODE  0
We reached our Goal 🏆
Number of steps 5
****************************************************
EPISODE  1
We reached our Goal 🏆
Number of steps 5
****************************************************
EPISODE  2
We reached our Goal 🏆
Number of steps 5
****************************************************
EPISODE  3
We reached our Goal 🏆
Number of steps 5
****************************************************
EPISODE  4
We reached our Goal 🏆
Number of steps 5
****************************************************
EPISODE  5
We reached our Goal 🏆
Number of steps 5
****************************************************
EPISODE  6
We reached our Goal 🏆
Number of steps 5
****************************************************
EPISODE  7
We reached our Goal 🏆
Number of steps 5
****************************************************
EPISODE  8
We reached our Goal 🏆
Number of steps 5
****************************************************
EPISODE  9


In [ ]:
q_learning(30000,epsilon)

Episode:  0
Episode:  1000
Episode:  2000
Episode:  3000
Episode:  4000
Episode:  5000
Episode:  6000
Episode:  7000
Episode:  8000
Episode:  9000
Episode:  10000
Episode:  11000
Episode:  12000
Episode:  13000
Episode:  14000
Episode:  15000
Episode:  16000
Episode:  17000
Episode:  18000
Episode:  19000
Episode:  20000
Episode:  21000
Episode:  22000
Episode:  23000
Episode:  24000
Episode:  25000
Episode:  26000
Episode:  27000
Episode:  28000
Episode:  29000
Average score over episodes: 29.467
[[0.73509189 0.77378094 0.77378094 0.73509189]
 [0.73509189 0.         0.81450625 0.77378094]
 [0.77378094 0.857375   0.77378094 0.81450625]
 [0.81450625 0.         0.77377476 0.77378093]
 [0.77378094 0.81450625 0.         0.73509189]
 [0.         0.         0.         0.        ]
 [0.         0.9025     0.         0.81450625]
 [0.         0.         0.         0.        ]
 [0.81450625 0.         0.857375   0.77378094]
 [0.81450625 0.9025     0.9025     0.        ]
 [0.857375   0.95       0. 

In [ ]:
test_agent(test_episodes)

****************************************************
EPISODE  0
We reached our Goal 🏆
Number of steps 5
****************************************************
EPISODE  1
We reached our Goal 🏆
Number of steps 5
****************************************************
EPISODE  2
We reached our Goal 🏆
Number of steps 5
****************************************************
EPISODE  3
We reached our Goal 🏆
Number of steps 5
****************************************************
EPISODE  4
We reached our Goal 🏆
Number of steps 5
****************************************************
EPISODE  5
We reached our Goal 🏆
Number of steps 5
****************************************************
EPISODE  6
We reached our Goal 🏆
Number of steps 5
****************************************************
EPISODE  7
We reached our Goal 🏆
Number of steps 5
****************************************************
EPISODE  8
We reached our Goal 🏆
Number of steps 5
****************************************************
EPISODE  9
